In [3]:
!pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.9 MB/s eta 0:00:00


In [16]:
!pip install optuna

In [26]:
import torch
import argparse
import numpy as np
import dill
import time
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import os
import torch.nn.functional as F
from collections import defaultdict

from models_initialize import GCN
from metrics import llprint, multi_label_metric, ddi_rate_score, get_n_params

torch.manual_seed(1203)
np.random.seed(1203)

model_name = 'GCN'
resume_name = ''

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--eval', action='store_true', default=False, help="eval mode")
parser.add_argument('--model_name', type=str, default=model_name, help="model name")
parser.add_argument('--resume_path', type=str, default=resume_name, help='resume path')
parser.add_argument('--ddi', action='store_true', default=False, help="using ddi")

args, unknown = parser.parse_known_args()  # Replace here
model_name = args.model_name
resume_name = args.resume_path

def main():
    if not os.path.exists(os.path.join("saved", model_name)):
        os.makedirs(os.path.join("saved", model_name))

    data_path = '/content/patient_records_final.pkl'
    voc_path = '/content/vocabulary_final.pkl'

    ehr_adj_path = '/content/ehr_adjacency_matrix_final.pkl'
    ddi_adj_path = '/content/ddi_adjacency_matrix_final.pkl'
    device = torch.device('cpu:0')

    ehr_adj = dill.load(open(ehr_adj_path, 'rb'))
    ddi_adj = dill.load(open(ddi_adj_path, 'rb'))
    data = dill.load(open(data_path, 'rb'))
    voc = dill.load(open(voc_path, 'rb'))

    diag_voc, pro_voc, med_voc = voc['diagnosis_vocabulary'], voc['procedure_vocabulary'], voc['medicine_vocabulary']

    split_point = int(len(data) * 2 / 3)
    data_train = data[:split_point]
    eval_len = int(len(data[split_point:]) / 2)
    data_test = data[split_point:split_point + eval_len]
    data_eval = data[split_point+eval_len:]

    # Define a set of hyperparameters to try out
    emb_dim_values = [32, 64, 128]
    dropout_values = [0.1, 0.2, 0.3, 0.4, 0.5]
    lr_values = [0.0001, 0.0002, 0.0005]
    epoch = 10
    Neg_Loss = args.ddi
    DDI_IN_MEM = args.ddi
    best_score = 0
    best_params = {}

    voc_size = (len(diag_voc.index_to_word), len(pro_voc.index_to_word), len(med_voc.index_to_word))

    # Try out all combinations of hyperparameters
    for emb_dim in emb_dim_values:
        for dropout in dropout_values:
            for lr in lr_values:
                print(f"Trying parameters: emb_dim={emb_dim}, dropout={dropout}, LR={lr}, EPOCH={epoch}")

                # Create a new instance of the model with the current set of parameters
                model = GCN(voc_size, ehr_adj, ddi_adj, emb_dim=emb_dim, dropout=dropout, device=torch.device('cpu:0'), ddi_in_memory=DDI_IN_MEM)
                #model.to(device)

                # Train the model and get its performance on the validation set
                optimizer = Adam(model.parameters(), lr=lr)
                criterion = CrossEntropyLoss()

                model.train()
                for current_epoch in range(epoch):
                    loss_record1 = []
                    prediction_loss_cnt = 0
                    neg_loss_cnt = 0
                    for step, input in enumerate(data_train):
                      #input = input.to(device)
                      for idx, adm in enumerate(input):
                          seq_input = input[:idx+1]
                          #seq_input = torch.tensor(seq_input).to(device)
                          loss1_target = np.zeros((1, voc_size[2]))
                          loss1_target[:, adm[2]] = 1
                          loss3_target = np.full((1, voc_size[2]), -1)
                          for idx, item in enumerate(adm[2]):
                              loss3_target[0][idx] = item

                          target_output1, batch_neg_loss = model(seq_input)

                          loss1 = F.binary_cross_entropy_with_logits(target_output1, torch.FloatTensor(loss1_target).to(device))
                          loss3 = F.multilabel_margin_loss(F.sigmoid(target_output1), torch.LongTensor(loss3_target).to(device))
                          if Neg_Loss:
                              target_output1 = F.sigmoid(target_output1).detach().cpu().numpy()[0]
                              target_output1[target_output1 >= 0.5] = 1
                              target_output1[target_output1 < 0.5] = 0
                              y_label = np.where(target_output1 == 1)[0]
                              current_ddi_rate = ddi_rate_score([[y_label]])
                              if current_ddi_rate <= TARGET_DDI:
                                  loss = 0.9 * loss1 + 0.01 * loss3
                                  prediction_loss_cnt += 1
                              else:
                                  rnd = np.exp((TARGET_DDI - current_ddi_rate)/T)
                                  if np.random.rand(1) < rnd:
                                      loss = batch_neg_loss
                                      neg_loss_cnt += 1
                                  else:
                                      loss = 0.9 * loss1 + 0.01 * loss3
                                      prediction_loss_cnt += 1
                          else:
                              loss = 0.9 * loss1 + 0.01 * loss3

                          optimizer.zero_grad()
                          loss.backward(retain_graph=True)
                          optimizer.step()

                          loss_record1.append(loss.item())

                llprint('\rTrain--Epoch: %d, Step: %d/%d, L_p cnt: %d, L_neg cnt: %d' % (epoch, step, len(data_train), prediction_loss_cnt, neg_loss_cnt))

                # Evaluate the model
                ddi_rate, ja, prauc, avg_p, avg_r, avg_f1 = eval(model, data_eval, voc_size, epoch)

                # Choose a score to optimize
                score = avg_f1

                # If the score on the validation set is the best we have seen, save the parameters
                if score > best_score:
                    best_score = score
                    best_params = {"emb_dim": emb_dim, "dropout": dropout, "LR": lr, "EPOCH": epoch}

    print("Best score: ", best_score)
    print("Best parameters: ", best_params)

if __name__ == '__main__':
    main()


Trying parameters: emb_dim=32, dropout=0.1, LR=0.0001, EPOCH=10
Train--Epoch: 10, Step: 3077/3078, L_p cnt: 0, L_neg cnt: 0
Eval--Epoch: 10, Step: 769/770	DDI Rate: 0.0828, Jaccard: 0.4244,  PRAUC: 0.6569, AVG_PRC: 0.6375, AVG_RECALL: 0.5572, AVG_F1: 0.5661
Trying parameters: emb_dim=32, dropout=0.1, LR=0.0002, EPOCH=10
Train--Epoch: 10, Step: 3077/3078, L_p cnt: 0, L_neg cnt: 0
Eval--Epoch: 10, Step: 769/770	DDI Rate: 0.0909, Jaccard: 0.4297,  PRAUC: 0.6671, AVG_PRC: 0.6578, AVG_RECALL: 0.5522, AVG_F1: 0.5698
Trying parameters: emb_dim=32, dropout=0.1, LR=0.0005, EPOCH=10
Train--Epoch: 10, Step: 3077/3078, L_p cnt: 0, L_neg cnt: 0
Eval--Epoch: 10, Step: 769/770	DDI Rate: 0.1037, Jaccard: 0.4404,  PRAUC: 0.6809, AVG_PRC: 0.6438, AVG_RECALL: 0.5783, AVG_F1: 0.5787
Trying parameters: emb_dim=32, dropout=0.2, LR=0.0001, EPOCH=10
Train--Epoch: 10, Step: 3077/3078, L_p cnt: 0, L_neg cnt: 0
Eval--Epoch: 10, Step: 769/770	DDI Rate: 0.0822, Jaccard: 0.4316,  PRAUC: 0.6580, AVG_PRC: 0.6418, AVG

Best parameters:  {'emb_dim': 128, 'dropout': 0.4, 'LR': 0.0005, 'EPOCH': 10}

In [12]:
def eval(model, data_eval, voc_size, epoch):
    # evaluate
    print('')
    model.eval()
    smm_record = []
    ja, prauc, avg_p, avg_r, avg_f1 = [[] for _ in range(5)]
    case_study = defaultdict(dict)
    med_cnt = 0
    visit_cnt = 0
    for step, input in enumerate(data_eval):
        y_gt = []
        y_pred = []
        y_pred_prob = []
        y_pred_label = []
        for adm_idx, adm in enumerate(input):

            target_output1 = model(input[:adm_idx+1])

            y_gt_tmp = np.zeros(voc_size[2])
            y_gt_tmp[adm[2]] = 1
            y_gt.append(y_gt_tmp)

            target_output1 = F.sigmoid(target_output1).detach().cpu().numpy()[0]
            y_pred_prob.append(target_output1)
            y_pred_tmp = target_output1.copy()
            y_pred_tmp[y_pred_tmp>=0.5] = 1
            y_pred_tmp[y_pred_tmp<0.5] = 0
            y_pred.append(y_pred_tmp)
            y_pred_label_tmp = np.where(y_pred_tmp == 1)[0]
            y_pred_label.append(sorted(y_pred_label_tmp))
            visit_cnt += 1
            med_cnt += len(y_pred_label_tmp)


        smm_record.append(y_pred_label)
        adm_ja, adm_prauc, adm_avg_p, adm_avg_r, adm_avg_f1 = multi_label_metric(np.array(y_gt), np.array(y_pred), np.array(y_pred_prob))
        case_study[adm_ja] = {'ja': adm_ja, 'patient': input, 'y_label': y_pred_label}

        ja.append(adm_ja)
        prauc.append(adm_prauc)
        avg_p.append(adm_avg_p)
        avg_r.append(adm_avg_r)
        avg_f1.append(adm_avg_f1)
        llprint('\rEval--Epoch: %d, Step: %d/%d' % (epoch, step, len(data_eval)))

    # ddi rate
    ddi_rate = ddi_rate_score(smm_record)

    llprint('\tDDI Rate: %.4f, Jaccard: %.4f,  PRAUC: %.4f, AVG_PRC: %.4f, AVG_RECALL: %.4f, AVG_F1: %.4f\n' % (
        ddi_rate, np.mean(ja), np.mean(prauc), np.mean(avg_p), np.mean(avg_r), np.mean(avg_f1)
    ))
    #dill.dump(obj=smm_record, file=open('content/gcnrecords.pkl', 'wb'))
    #dill.dump(case_study, open(os.path.join('saved', model_name, 'case_study.pkl'), 'wb'))

    # print('avg med', med_cnt / visit_cnt)

    return ddi_rate, np.mean(ja), np.mean(prauc), np.mean(avg_p), np.mean(avg_r), np.mean(avg_f1)

In [13]:

import torch
import argparse
import numpy as np
import dill
import time
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import os
import torch.nn.functional as F
from collections import defaultdict

from models_initialize import GCN
from metrics import llprint, multi_label_metric, ddi_rate_score, get_n_params

torch.manual_seed(1203)
np.random.seed(1203)

model_name = 'GCN'
resume_name = ''

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--eval', action='store_true', default=False, help="eval mode")
parser.add_argument('--model_name', type=str, default=model_name, help="model name")
parser.add_argument('--resume_path', type=str, default=resume_name, help='resume path')
parser.add_argument('--ddi', action='store_true', default=False, help="using ddi")

args, unknown = parser.parse_known_args()  # Replace here
model_name = args.model_name
resume_name = args.resume_path

def main():
    if not os.path.exists(os.path.join("saved", model_name)):
        os.makedirs(os.path.join("saved", model_name))

    data_path = '/content/patient_records_final.pkl'
    voc_path = '/content/vocabulary_final.pkl'

    ehr_adj_path = '/content/ehr_adjacency_matrix_final.pkl'
    ddi_adj_path = '/content/ddi_adjacency_matrix_final.pkl'
    device = torch.device('cpu:0')

    ehr_adj = dill.load(open(ehr_adj_path, 'rb'))
    ddi_adj = dill.load(open(ddi_adj_path, 'rb'))
    data = dill.load(open(data_path, 'rb'))
    voc = dill.load(open(voc_path, 'rb'))

    diag_voc, pro_voc, med_voc = voc['diagnosis_vocabulary'], voc['procedure_vocabulary'], voc['medicine_vocabulary']

    split_point = int(len(data) * 2 / 3)
    data_train = data[:split_point]
    eval_len = int(len(data[split_point:]) / 2)
    data_test = data[split_point:split_point + eval_len]
    data_eval = data[split_point+eval_len:]

    #Optimal hyperparameters
    emb_dim = 128
    dropout = 0.4
    lr = 0.0005
    epoch = 10
    Neg_Loss = args.ddi
    DDI_IN_MEM = args.ddi
    best_score = 0
    best_params = {}

    voc_size = (len(diag_voc.index_to_word), len(pro_voc.index_to_word), len(med_voc.index_to_word))

    # Create a new instance of the model with the current set of parameters
    model = GCN(voc_size, ehr_adj, ddi_adj, emb_dim=emb_dim, dropout=dropout, device=torch.device('cpu:0'), ddi_in_memory=DDI_IN_MEM)
    #model.to(device)

    # Train the model and get its performance on the validation set
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = CrossEntropyLoss()

    model.train()
    for current_epoch in range(epoch):
        loss_record1 = []
        prediction_loss_cnt = 0
        neg_loss_cnt = 0
        for step, input in enumerate(data_train):
          #input = input.to(device)
          for idx, adm in enumerate(input):
              seq_input = input[:idx+1]
              #seq_input = torch.tensor(seq_input).to(device)
              loss1_target = np.zeros((1, voc_size[2]))
              loss1_target[:, adm[2]] = 1
              loss3_target = np.full((1, voc_size[2]), -1)
              for idx, item in enumerate(adm[2]):
                  loss3_target[0][idx] = item

              target_output1, batch_neg_loss = model(seq_input)

              loss1 = F.binary_cross_entropy_with_logits(target_output1, torch.FloatTensor(loss1_target).to(device))
              loss3 = F.multilabel_margin_loss(F.sigmoid(target_output1), torch.LongTensor(loss3_target).to(device))
              if Neg_Loss:
                  target_output1 = F.sigmoid(target_output1).detach().cpu().numpy()[0]
                  target_output1[target_output1 >= 0.5] = 1
                  target_output1[target_output1 < 0.5] = 0
                  y_label = np.where(target_output1 == 1)[0]
                  current_ddi_rate = ddi_rate_score([[y_label]])
                  if current_ddi_rate <= TARGET_DDI:
                      loss = 0.9 * loss1 + 0.01 * loss3
                      prediction_loss_cnt += 1
                  else:
                      rnd = np.exp((TARGET_DDI - current_ddi_rate)/T)
                      if np.random.rand(1) < rnd:
                          loss = batch_neg_loss
                          neg_loss_cnt += 1
                      else:
                          loss = 0.9 * loss1 + 0.01 * loss3
                          prediction_loss_cnt += 1
              else:
                  loss = 0.9 * loss1 + 0.01 * loss3

              optimizer.zero_grad()
              loss.backward(retain_graph=True)
              optimizer.step()

              loss_record1.append(loss.item())

    llprint('\rTrain--Epoch: %d, Step: %d/%d, L_p cnt: %d, L_neg cnt: %d' % (epoch, step, len(data_train), prediction_loss_cnt, neg_loss_cnt))

    # Evaluate the model
    ddi_rate, ja, prauc, avg_p, avg_r, avg_f1 = eval(model, data_eval, voc_size, epoch)
    llprint('\EVAL--Epoch: %d, Step: %d/%d, L_p cnt: %d, L_neg cnt: %d' % (epoch, step, len(data_train), prediction_loss_cnt, neg_loss_cnt))

    ddi_rate, ja, prauc, avg_p, avg_r, avg_f1 = eval(model, data_test, voc_size, 0)
    llprint('\TEST--Epoch: %d, Step: %d/%d, L_p cnt: %d, L_neg cnt: %d' % (epoch, step, len(data_train), prediction_loss_cnt, neg_loss_cnt))

if __name__ == '__main__':
    main()


Train--Epoch: 10, Step: 3077/3078, L_p cnt: 0, L_neg cnt: 0
Eval--Epoch: 10, Step: 769/770	DDI Rate: 0.1002, Jaccard: 0.4586,  PRAUC: 0.6906, AVG_PRC: 0.6347, AVG_RECALL: 0.6208, AVG_F1: 0.5988
\EVAL--Epoch: 10, Step: 3077/3078, L_p cnt: 0, L_neg cnt: 0
Eval--Epoch: 0, Step: 769/770	DDI Rate: 0.0993, Jaccard: 0.4550,  PRAUC: 0.6943, AVG_PRC: 0.6525, AVG_RECALL: 0.5977, AVG_F1: 0.5982
\TEST--Epoch: 10, Step: 3077/3078, L_p cnt: 0, L_neg cnt: 0